# Project Work - IRM24
## Jürgen Aumayr & Natalia Trudova

## Vergleich der Modelle

### **Precision (Genauigkeit der positiven Vorhersagen)**
Precision misst den Anteil der tatsächlich korrekten positiven Vorhersagen an allen als positiv vorhergesagten Fällen.

Formel:

$$
\text{Precision} = \frac{TP}{TP + FP}
$$
 
- **TP (True Positives):** Korrekt als positiv klassifizierte Fälle
- **FP (False Positives):** Fälschlicherweise als positiv klassifizierte Fälle

Eine hohe Precision bedeutet, dass das Modell selten fälschlicherweise positive Vorhersagen trifft. Das ist besonders wichtig, wenn die Kosten für Fehlalarme (False Positives) hoch sind.


### **Recall (Sensitivität, Trefferquote)**
Recall misst den Anteil der korrekt erkannten positiven Fälle an allen tatsächlich positiven Fällen.

Formel:

$$
\text{Recall} = \frac{TP}{TP + FN}
$$

- **TP (True Positives):** Korrekt als positiv klassifizierte Fälle
- **FN (False Negatives):** Tatsächlich positive Fälle, die das Modell nicht erkannt hat

Ein hoher Recall bedeutet, dass das Modell die meisten tatsächlichen Positiven findet. Das ist wichtig, wenn das Verpassen eines positiven Falls (False Negative) schwerwiegende Folgen hat.


### **F1-Score (Harmonisches Mittel von Precision und Recall)**
Der F1-Score kombiniert Precision und Recall zu einer einzigen Metrik, indem er ihr harmonisches Mittel berechnet.

Formel:

$$
\text{F1-Score} = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}
$$

Der F1-Score ist besonders nützlich bei unausgeglichenen Datensätzen, da er nur dann hoch ist, wenn sowohl Precision als auch Recall hoch sind. Er ist die Standardmetrik, wenn ein ausgewogenes Verhältnis zwischen Precision und Recall wichtig ist.


### **Support (Anzahl der wahren Instanzen pro Klasse)**
Support gibt an, wie viele tatsächliche Beispiele es pro Klasse im Datensatz gibt.

Support selbst ist keine Leistungsmetrik, sondern gibt Kontext: Ein hoher oder niedriger Support kann die Aussagekraft von Precision, Recall und F1-Score beeinflussen, insbesondere bei stark unbalancierten Klassen.

In [1]:
# Notwendige Bibliotheken importieren 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import plot_tree

# Optimierte Datentypen definieren
dtypes = {
    'PassengerId': 'int32',  # 32-bit Integer für IDs ist ausreichend
    'Survived': 'int8',      # 8-bit Integer für binäre Werte (0/1)
    'Pclass': 'int8',        # 8-bit Integer für kleine Kategorien (1-3)
    'Name': 'str',           # String für Namen
    'Sex': 'category',       # Kategorie-Typ für höhere Effizienz bei wiederholten Werten
    'Age': 'float32',        # 32-bit Float statt Standard 64-bit
    'SibSp': 'int8',         # 8-bit Integer für kleine Zahlen
    'Parch': 'int8',         # 8-bit Integer für kleine Zahlen
    'Ticket': 'str',         # String für Ticketnummern
    'Fare': 'float32',       # 32-bit Float für Fahrpreise
    'Cabin': 'str',          # String für Kabinennummern
    'Embarked': 'category'   # Kategorie-Typ für Einschiffungshäfen (S, C, Q)
}

# Daten mit optimierten Datentypen einlesen 
train_df = pd.read_csv('data/train.csv', dtype=dtypes)

In [2]:
# Feature Engineering
def preprocess_features(df):
    """
    Bereitet die Titanic-Daten für das Modelltraining vor und
    erzeugt optimierte Features aus den Rohdaten.
    """
    # Kopie erstellen, um Originaldaten nicht zu verändern
    dataset = df.copy()
    
    # Geschlecht in numerisch konvertieren
    dataset['Sex'] = dataset['Sex'].map({'female': 0, 'male': 1})
    
    # Fehlende Alterswerte mit Median füllen - nach Klasse und Geschlecht gruppiert
    age_median = dataset.groupby(['Pclass', 'Sex'])['Age'].transform('median')
    dataset['Age'] = dataset['Age'].fillna(age_median)

    # NaN-Werte durch den Median ersetzen
    dataset['Fare'] = dataset['Fare'].fillna(dataset['Fare'].median())
    
    # Alter in Kategorien einteilen
    dataset['Age'] = pd.cut(
        dataset['Age'], 
        bins=[0, 18, 60, np.inf], 
        labels=[1, 2, 3]
    ).astype('int8')
    
    # Fahrpreise in Quartile einteilen für bessere Vergleichbarkeit
    dataset['Fare'] = pd.qcut(
        dataset['Fare'], 
        q=4, 
        labels=[0, 1, 2, 3],
        duplicates='drop'  # Verhindert Fehler bei doppelten Grenzwerten
    ).astype('int8')
    
    # Einschiffungshafen in numerische Werte umwandeln
    dataset['Embarked'] = dataset['Embarked'].fillna(dataset['Embarked'].mode()[0])
    dataset['Embarked'] = dataset['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype('int8')
    
    # Binäres Merkmal: Hat der Passagier eine Kabinennummer oder nicht?
    dataset['Has_Cabin'] = (dataset['Cabin'].notna()).astype('int8')
    
    # Berechnung der Familiengröße und Alleinreisenden-Status
    dataset['FamilySize'] = (dataset['SibSp'] + dataset['Parch'] + 1).astype('int8')
    dataset['IsAlone'] = (dataset['FamilySize'] == 1).astype('int8')
    
    # Titel aus Namen extrahieren mit regulärem Ausdruck - effizienter als Split
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    
    # Titel-Mapping für Kategorisierung
    title_mapping = {
        "Mr": 1, "Master": 2, "Mrs": 3, "Miss": 4, "Rare": 5
    }
    
    # Seltenere Titel zusammenfassen, um Overfitting zu vermeiden
    rare_titles = ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 
                   'Major', 'Rev', 'Sir', 'Jonkheer', 'Mlle', 'Mme', 'Ms']
    dataset['Title'] = dataset['Title'].replace(rare_titles, 'Rare')
    dataset['Title'] = dataset['Title'].map(title_mapping).fillna(5).astype('int8')
    
    return dataset

# Daten vorverarbeiten und relevante Features auswählen
processed_df = preprocess_features(train_df)
features_df = processed_df[['Survived', 'Pclass', 'Sex', 'Age', 'Parch', 
                            'Fare', 'Embarked', 'Has_Cabin', 'FamilySize', 
                            'IsAlone', 'Title']]

In [3]:
# Features und Zielvariable trennen
X = features_df.drop('Survived', axis=1)
y = features_df['Survived']

# Daten in Trainings- und Validierungssets aufteilen mit stratifizierter Stichprobe
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [4]:
# Decision Tree Training
# Hyperparameter-Grid für automatische Optimierung
param_grid = {
    'max_depth': [3, 4, 5, 6],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

# Parallelisierte Suche mit 5-facher Kreuzvalidierung
grid_search = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid,
    cv=5,
    n_jobs=-1,  # Nutzt alle verfügbaren CPU-Kerne
    scoring='accuracy'
)

# Modell trainieren
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 4, 5, 6],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='accuracy')

In [5]:
# Random Forests Training
# Hyperparameter-Grid für Random Forest
param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [4, 6],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2],
}

# GridSearchCV für Hyperparameter-Tuning verwenden
grid_search_rf = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    cv=5,
    n_jobs=-1,
)

# Modell trainieren
grid_search_rf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [4, 6], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200]})

In [6]:
# Gradient Boosting Training

# Gradient Boosting Parameter-Space
gb_params = {
    'learning_rate': [0.05, 0.1],
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'min_samples_split': [10, 20]
}

# Grid Search mit Early Stopping
gb_search = GridSearchCV(
    GradientBoostingClassifier(
        subsample=0.8,
        validation_fraction=0.2,
        n_iter_no_change=5,
        random_state=42
    ),
    param_grid=gb_params,
    scoring='accuracy',
    cv=5,
    n_jobs=-1
)

gb_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=GradientBoostingClassifier(n_iter_no_change=5,
                                                  random_state=42,
                                                  subsample=0.8,
                                                  validation_fraction=0.2),
             n_jobs=-1,
             param_grid={'learning_rate': [0.05, 0.1], 'max_depth': [3, 5],
                         'min_samples_split': [10, 20],
                         'n_estimators': [100, 200]},
             scoring='accuracy')

In [13]:
# Comparision

# Decision Tree
best_model = grid_search.best_estimator_
val_pred = best_model.predict(X_val)
val_acc_dt = accuracy_score(y_val, val_pred)
print("--------------------Decision Tree---------------------")
print(f"Validierungs-Genauigkeit: {val_acc_dt:.4f}")
print("\nKlassifikationsbericht:")
print(classification_report(y_val, val_pred, target_names=['Gestorben', 'Überlebt']))

# Random Forests
best_rf_model = grid_search_rf.best_estimator_
val_predictions = best_rf_model.predict(X_val)
val_acc_rf = accuracy_score(y_val, val_predictions)
print("-------------------Random Forests---------------------")
print(f"Validierungsgenauigkeit: {val_acc_rf:.4f}")
print("\nKlassifikationsbericht:")
print(classification_report(y_val, val_predictions, target_names=['Gestorben', 'Überlebt']))

# Gradient Boosting
best_gb = gb_search.best_estimator_
val_preds = best_gb.predict(X_val)
val_acc_gb = accuracy_score(y_val, val_preds)
print("------------------Gradient Boosting-------------------")
print(f"Validierungsgenauigkeit: {accuracy_score(y_val, val_preds):.4f}")
print("\nKlassifikationsbericht:")
print(classification_report(y_val, val_preds, target_names=['Gestorben', 'Überlebt']))

--------------------Decision Tree---------------------
Validierungs-Genauigkeit: 0.7933

Klassifikationsbericht:
              precision    recall  f1-score   support

   Gestorben       0.80      0.88      0.84       110
    Überlebt       0.78      0.65      0.71        69

    accuracy                           0.79       179
   macro avg       0.79      0.77      0.77       179
weighted avg       0.79      0.79      0.79       179

-------------------Random Forests---------------------
Validierungsgenauigkeit: 0.7989

Klassifikationsbericht:
              precision    recall  f1-score   support

   Gestorben       0.81      0.87      0.84       110
    Überlebt       0.77      0.68      0.72        69

    accuracy                           0.80       179
   macro avg       0.79      0.78      0.78       179
weighted avg       0.80      0.80      0.80       179

------------------Gradient Boosting-------------------
Validierungsgenauigkeit: 0.8101

Klassifikationsbericht:
         